# Automação para Agendamento de Carregamento

### Imports

In [1]:
import pandas as pd
import getpass
import keyring
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

### Funções

In [2]:
XPATH_usuario = '/html/body/div/div/div[1]/div[1]/div/form/div[1]/div[2]/div/input'
XPATH_senha = '/html/body/div/div/div[1]/div[1]/div/form/div[1]/div[3]/div/input'
XPATH_acessar = '/html/body/div/div/div[1]/div[1]/div/form/div[1]/div[5]/input'

# função para logar no site
def f_log():    
    # guardando usuario e senha de login vibra em variaveis
    usuario = '1596189'
    senha = keyring.get_password(service_name='login_br', username='1596189')
    
    # logando no sistema vibra
    driver.find_element(By.XPATH, XPATH_usuario).send_keys(usuario) # preenche usuario
    sleep(1)
    driver.find_element(By.XPATH, XPATH_senha).send_keys(senha) # preenche senha
    sleep(1)
    driver.find_element(By.XPATH, XPATH_acessar).click() # clica acessar
    if driver.current_url == 'https://cn.vibraenergia.com.br/homecn/':
        print('login OK')
    else:
        print('login falhou')

In [3]:
XPATH_criar_programacao = '/html/body/div[3]/div[5]/div/div/form/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div/div/div[2]/div/a[1]'
XPATH_pesquisar_remessas = '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div/div/div/span[2]/input'
XPATH_placa = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/span/div/div[2]/input[1]'
XPATH_buscar_placa = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/span/div/div[2]/input[2]'
XPATH_lista_transport = '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div[2]/div/div/select'

def criar_programacao():
    driver.find_element(By.XPATH, XPATH_criar_programacao).click() # clica em criar programação
    sleep(1)
    driver.find_element(By.XPATH, XPATH_pesquisar_remessas).click() # pesquisa remessas
    sleep(1)
    driver.find_element(By.XPATH, XPATH_placa).send_keys('PGS3G71') # preenche a placa
    sleep(1)
    driver.find_element(By.XPATH, XPATH_buscar_placa).click() # clica ok na placa
    sleep(1)
    select = Select(driver.find_element(By.XPATH, XPATH_lista_transport)) # procura a lista suspensa
    select.select_by_visible_text('06.928.937/0001-08 - PETROBARROS TRANSPORTADORA LTDA. - 0010121115') # seleciona transportador

In [4]:
# XPATH_lista_horarios = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[2]/center/div/div[2]/div/center/select'

# def busca_horario():
#     # encontra o calendario
#     dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
#     dias_ext = []
    
#     # extrai os dias do calendario
#     for i in range(len(dias)): 
#         dias_ext.append(dias[i].text)
    
#     sleep(1)
#     # clica no dia desejado
#     for i in range(len(dias_ext)): 
#         if dias_ext[i] == dia_desejado:
#             dias[dias_ext.index(dia_desejado)].click()    
    
#     sleep(1)
    
#     horarios_encontrados = []
    
#     # o bloco try e except a seguir está sendo responsável por verificar se a lista suspensa de horarios está aparecendo.
#     # se a lista suspensa estiver aparecendo o código continua. Se não, a função é chamada do início. Não é o jeito mais
#     # elegante de fazer isso, mas acho que está funcionando.
    
#     try:
#         select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
                
#     except EC.NoSuchElementException:
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
#         # clica em avancar para recarregar a pagina
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario()
    
#     select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
#     for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
#         if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
#             horarios_encontrados.append(select_horario.options[i].text)
#     if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
#         # clica em avancar para recarregar a paginar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario()
        
#     return horarios_encontrados

In [5]:
# # essa funcao foi reescrita para contornar um problema de instabilidade no site da vibra

# def busca_horario():
#     # encontra o calendario
#     try:
#         dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
#     except EC.NoSuchElementException:
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
#         # clica em avancar para recarregar a pagina
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario()
    
#     dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
#     dias_ext = []
    
#     # extrai os dias do calendario
#     for i in range(len(dias)): 
#         dias_ext.append(dias[i].text)
    
#     sleep(1)
#     # clica no dia desejado
#     for i in range(len(dias_ext)): 
#         if dias_ext[i] == dia_desejado:
#             dias[dias_ext.index(dia_desejado)].click()    
    
#     sleep(1)
    
#     horarios_encontrados = []
    
#     # o bloco try e except a seguir está sendo responsável por verificar se a lista suspensa de horarios está aparecendo.
#     # se a lista suspensa estiver aparecendo o código continua. Se não, a função é chamada do início. Não é o jeito mais
#     # elegante de fazer isso, mas acho que está funcionando.
    
#     try:
#         select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
                
#     except EC.NoSuchElementException:
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
#         sleep(1)
#         # clica em avancar para recarregar a pagina
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario()
    
#     select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
#     for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
#         if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
#             horarios_encontrados.append(select_horario.options[i].text)
#     if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a[1]').click()
#         sleep(1)
#         # clica em avancar para recarregar a paginar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario()
        
#     return horarios_encontrados

In [6]:
# essa versão da função busca_horario está atualizada para ao invés de voltar e avançar a pagina apenas atualizar a pagina

# XPATH_lista_horarios = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[2]/center/div/div[2]/div/center/select'

# def busca_horario():
#     # o bloco try e except a seguir procura o calendario, caso n encontre ele atualiza a pagina e chama a funcao de novo
#     try:
#         dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
#     except EC.NoSuchElementException:
#         sleep(5)
#         driver.refresh()
#         busca_horario()
    
#     sleep(2)
#     dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
#     dias_ext = []
    
#     # extrai os dias do calendario para comparacao com o input de dia desejado
#     for i in range(len(dias)): 
#         dias_ext.append(dias[i].text)
    
#     sleep(2)
#     # clica no dia desejado
#     for i in range(len(dias_ext)): 
#         if dias_ext[i] == dia_desejado:
#             dias[dias_ext.index(dia_desejado)].click()    
    
#     sleep(2)
#     horarios_encontrados = []
    
#     # o bloco try e except a seguir procura a lista suspensa de horarios,
#     # caso n encontre ele atualiza a pagina e chama a funcao de novo
    
#     try:
#         select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))              
#     except EC.NoSuchElementException:
#         sleep(5)
#         driver.refresh()
#         busca_horario()
    
#     sleep(2)
#     # a linha abaixo está duplicada (ela ja aparece no bloco try anterior), por isso irei comenta-la
#     # n irei apaga-la ainda pois ainda falta testar
#     # select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
#     for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
#         if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
#             horarios_encontrados.append(select_horario.options[i].text)
#     if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
#         sleep(5)
#         driver.refresh() # atualiza a pagina
#         busca_horario()
        
#     return horarios_encontrados

In [7]:
# # essa versão da função busca_horario está atualizada para ao invés de voltar e avançar a pagina apenas atualizar a pagina
# # mas sem os blocos try e except que estavam sendo utilizados para evitar problemas de carregamento da pagina

# XPATH_lista_horarios = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[2]/center/div/div[2]/div/center/select'

# def busca_horario():
#     sleep(2)
#     dias = driver.find_element(By.CLASS_NAME, 'datepicker-days').find_elements(By.CLASS_NAME, 'day')
#     dias_ext = []
    
#     # extrai os dias do calendario para comparacao com o input de dia desejado
#     for i in range(len(dias)): 
#         dias_ext.append(dias[i].text)
    
#     sleep(2)
#     # clica no dia desejado
#     for i in range(len(dias_ext)): 
#         if dias_ext[i] == dia_desejado:
#             dias[dias_ext.index(dia_desejado)].click()    
    
#     sleep(2)
#     horarios_encontrados = []
    
#     # o bloco try e except a seguir procura a lista suspensa de horarios,
#     # caso n encontre ele atualiza a pagina e chama a funcao de novo

#     select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))              
    
#     sleep(2)
#     # a linha abaixo está duplicada (ela ja aparece no bloco try anterior), por isso irei comenta-la
#     # n irei apaga-la ainda pois ainda falta testar
#     # select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
#     for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
#         if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
#             horarios_encontrados.append(select_horario.options[i].text)
#     if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
#         sleep(5)
#         driver.refresh() # atualiza a pagina
#         busca_horario()
        
#     return horarios_encontrados

In [8]:
# def busca_horario_28_08():
#     # encontra o calendario
#     try:
#         # clica no dia 28-08
#         sleep(2)
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div[2]/div[1]/center/div/div[2]/div/div/div/div[1]/table/tbody/tr[5]/td[4]').click()
#     except EC.NoSuchElementException:
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
#         # clica em avancar para recarregar a pagina
#         sleep(2)
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario_28_08()  
    
#     sleep(1)
    
#     horarios_encontrados = []
    
#     # o bloco try e except a seguir está sendo responsável por verificar se a lista suspensa de horarios está aparecendo.
#     # se a lista suspensa estiver aparecendo o código continua. Se não, a função é chamada do início. Não é o jeito mais
#     # elegante de fazer isso, mas acho que está funcionando.
    
#     try:
#         sleep(2)
#         select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
                
#     except EC.NoSuchElementException:
#         sleep(5)
#         # clica em voltar
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[4]/div/div/div/span/a').click()
#         sleep(2)
#         # clica em avancar para recarregar a pagina
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#         busca_horario_28_08()
    
#     sleep(2)
#     select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
#     for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
#         if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
#             horarios_encontrados.append(select_horario.options[i].text)
#     if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
#         sleep(5)
#         # clica no dia atual
#         try:
#             driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[1]/center/div/div[2]/div/div/div/div[1]/table/tbody/tr[5]/td[3]').click()
#             sleep(2)
#             busca_horario_28_08()
#         except:
#             # clica em voltar
#             sleep(2)
#             driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div/div/div/span/a[1]').click()
#             sleep(2)
#             # clica em avancar para recarregar a pagina
#             driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[5]/div/div/div/span/a[2]').click()
#             busca_horario_28_08()
#     return horarios_encontrados

In [9]:
# XPATH_lista_horarios = '/html/body/div[2]/div[5]/div/div/div/form/div[2]/div[2]/div[2]/center/div/div[2]/div/center/select'

# def busca_horario_26_09():
#     # clica no dia 26/09
#     sleep(1)
#     try:
#         driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/div/div/div/form/div[3]/div[2]/div[1]/center/div/div[2]/div/div/div/div[1]/table/tbody/tr[5]/td[5]').click()
#     except EC.NoSuchElementException:
#         sleep(5)
#         print('não clicou no dia 26')
#         busca_horario_26_09()
    
#     horarios_encontrados = []
    
#     sleep(2)
#     try:
#         select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
#     except EC.NoSuchElementException:
#         sleep(5)
#         print('não encontrou a lista suspensa de horarios')
#         busca_horario_26_09()
        
#     select_horario = Select(driver.find_element(By.XPATH,XPATH_lista_horarios))
    
#     for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
#         if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
#             horarios_encontrados.append(select_horario.options[i].text)
#     if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
#         sleep(5)
#         # refresh para atualizar horarios
#         driver.refresh()
#         sleep(2)
#         busca_horario_26_09()
#     return horarios_encontrados

In [10]:
# a função deve pedir as seguintes variaveis já no inicio: dia_desejado, horario_inicial, horario_final
# 1o. identificar o dia selecionado ao entrar na pag de escolha de dia e horario e armazenar essa informacao
# 2o. se o dia a ser buscado for o dia atual já pula para a procura de horario. Se não, antes clica no dia a ser buscado

def informa_data_e_hora():
    
    dia_desejado = input('Dia desejado: ')
    horario_inicial = int(input('Horário Inicial (apenas números):'))
    horario_final = int(input('Horário Final (apenas números):'))
    
    return dia_desejado, horario_inicial, horario_final

def busca_horario_v2():
    
    dia_ativo = driver.find_element(By.CLASS_NAME, 'active').text
    sleep(2)
    
    horarios_encontrados = []
    
    if dia_ativo == dia_desejado: # se o dia desejado for o dia atual
        try:
            select_horario = Select(driver.find_element(By.ID,'combo_horarios'))
        
            for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
                if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
                    horarios_encontrados.append(select_horario.options[i].text)
                    select_horario.select_by_visible_text(select_horario.options[i].text)
                    driver.find_element(By.LINK_TEXT, 'Finalizar').click() ### ESSA LINHA FINALIZA A PROGRAMAÇÃO ###
            if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
                sleep(5)
                driver.find_element(By.CLASS_NAME, 'today').click()
                sleep(2)
                driver.find_element(By.CLASS_NAME, 'today').click()
                busca_horario_v2()
            else:
                return horarios_encontrados
            
        except EC.NoSuchElementException:
            driver.find_element(By.CLASS_NAME, 'today').click()
            sleep(2)
            driver.find_element(By.CLASS_NAME, 'today').click()
            busca_horario_v2()
    
    else: # escrever código para procurar horario neste dia
        lista_dias = driver.find_elements(By.CLASS_NAME, 'day')
        
        for i in range(len(lista_dias)):
            try:
                if lista_dias[i].text == dia_desejado:
                    lista_dias[i].click()
            except EC.StaleElementReferenceException:
                pass
        
        try:
            select_horario = Select(driver.find_element(By.ID,'combo_horarios'))
        
            for i in range(len(select_horario.options)): # guarda os horários disponiveis na variavel horarios_encontrados
                if int(select_horario.options[i].text[13:15]) >= horario_inicial and int(select_horario.options[i].text[13:15]) <= horario_final:
                    horarios_encontrados.append(select_horario.options[i].text)
                    select_horario.select_by_visible_text(select_horario.options[i].text)
                    driver.find_element(By.LINK_TEXT, 'Finalizar').click() ### ESSA LINHA FINALIZA A PROGRAMAÇÃO ###
            if len(horarios_encontrados) == 0: # se não encontrar nenhum horário dentro das especificações, a função é chamada de novo
                sleep(5)
                # refresh para atualizar horarios
                driver.find_element(By.CLASS_NAME, 'today').click()
                sleep(2)
                driver.find_element(By.CLASS_NAME, 'today').click()
                busca_horario_v2()                
        except EC.NoSuchElementException:
            sleep(2)
            driver.find_element(By.CLASS_NAME, 'today').click()
            busca_horario_v2()
            
        return horarios_encontrados        

In [11]:
remetente = 'carregamentopetrobarros@gmail.com'
senha = keyring.get_password(service_name='carregamento_petrobarros', username='carregamentopetrobarros@gmail.com')
destinatario = ['gustavossbarros@hotmail.com', 'toninhobarrosf@hotmail.com']
assunto = 'Novo Horario de Carregamento'

def envia_email(sender_email, password, recipient, subject, body):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = ", ".join(recipient)  # Junta os destinatários em uma string separada por vírgulas
    message['Subject'] = subject

    message.attach(MIMEText(body, 'plain'))

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, password)
        text = message.as_string()
        server.sendmail(sender_email, recipient, text)

### Inicializando Google Chrome

In [13]:
# se essa parte do código ficar dentro de uma função o programa não consegue manter o chrome aberto

service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

url = 'https://cn.vibraenergia.com.br/login/'

driver.get(url)

### Rodando as funções

In [19]:
f_log()

login OK


In [20]:
criar_programacao()

In [16]:
# dia_desejado = input(prompt='Dia Desejado: ') # n inteiro pois esse valor é comparado com uma string na pagina da vibra
# horario_inicial = int(input(prompt='Horário Inicial (apenas número da hora): '))
# horario_final = int(input(prompt='Horário Final (apenas número da hora): '))
# horarios = busca_horario_26_09()

# corpo_do_email = f'''Encontrei os seguintes horários de carregamento.
# {horarios}
# --------------------------------
# EMAIL AUTOMÁTICO - NÃO RESPONDER
# '''
# envia_email(remetente, senha, destinatario, assunto, corpo_do_email)

In [18]:
# testando as novas funções

dia_desejado, horario_inicial, horario_final = informa_data_e_hora()

horarios = busca_horario_v2()

corpo_do_email = f'''Encontrei os seguintes horários de carregamento.
{horarios}
--------------------------------
EMAIL AUTOMÁTICO - NÃO RESPONDER
'''
envia_email(remetente, senha, destinatario, assunto, corpo_do_email)

Dia desejado: 27
Horário Inicial (apenas números):12
Horário Final (apenas números):16


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF618F16CB5+28821]
	(No symbol) [0x00007FF618E83840]
	(No symbol) [0x00007FF618D2578A]
	(No symbol) [0x00007FF618D39D94]
	(No symbol) [0x00007FF618D38844]
	(No symbol) [0x00007FF618D2CF91]
	(No symbol) [0x00007FF618D2ADB5]
	(No symbol) [0x00007FF618D2EFCC]
	(No symbol) [0x00007FF618D2F09F]
	(No symbol) [0x00007FF618D78C6A]
	(No symbol) [0x00007FF618D794AC]
	(No symbol) [0x00007FF618D6C9FC]
	(No symbol) [0x00007FF618D9F33F]
	(No symbol) [0x00007FF618D6C3F6]
	(No symbol) [0x00007FF618D9F510]
	(No symbol) [0x00007FF618DBF412]
	(No symbol) [0x00007FF618D9F0A3]
	(No symbol) [0x00007FF618D6A778]
	(No symbol) [0x00007FF618D6B8E1]
	GetHandleVerifier [0x00007FF61924FCAD+3408013]

In [ ]:
# # essa é a lógica para clicar no dia desejado

# lista_dias = driver.find_elements(By.CLASS_NAME, 'day')
# for i in range(len(lista_dias)):
#     try:
#         if lista_dias[i].text == dia_desejado:
#             lista_dias[i].click()
#     except EC.StaleElementReferenceException:
#         pass

- driver.refresh() para atualizar
- driver.find_element(By.LINK_TEXT, 'texto') para encontrar elementos baseado em texto
- driver.